In [13]:
import re
pat4U =  '::|\n'
pat4M=  '::|\n'
uData= open("users.dat","r")
mData=open("movies.dat","r",encoding='cp1252')
rData=open("ratings.dat","r")
u_list= re.split(pat4U,uData.read())
m_list=re.split(pat4M,mData.read())
r_list=re.split(pat4M,rData.read())

users={}
movies={}
ratings={}
for i in range(0,len(u_list)-5,5):
    users[u_list[i]]={}
    users[u_list[i]]['Gender']=u_list[i+1]
    users[u_list[i]]['Occupation']=u_list[i+3]
for i in range(0,len(m_list)-3,3):
    movies[m_list[i]]={}
    movies[m_list[i]]['Title']=m_list[i+1]
for i in range(0,len(r_list)-4,4):
    ratings[r_list[i]]={}
    ratings[r_list[i]][r_list[i+1]]=r_list[i+2]

ratings

{'1': {'1246': '4'},
 '2': {'1247': '5'},
 '3': {'2081': '4'},
 '4': {'1240': '5'},
 '5': {'714': '4'},
 '6': {'34': '4'},
 '7': {'3418': '3'},
 '8': {'36': '4'},
 '9': {'428': '3'},
 '10': {'2049': '5'},
 '11': {'1244': '4'},
 '12': {'1247': '3'},
 '13': {'2046': '4'},
 '14': {'2976': '3'},
 '15': {'2115': '4'},
 '16': {'2975': '5'},
 '17': {'2985': '5'},
 '18': {'1246': '5'},
 '19': {'3928': '2'},
 '20': {'1240': '4'},
 '21': {'3774': '1'},
 '22': {'1247': '4'},
 '23': {'2985': '5'},
 '24': {'1247': '4'},
 '25': {'1676': '4'},
 '26': {'1247': '3'},
 '27': {'1097': '5'},
 '28': {'1247': '5'},
 '29': {'1242': '2'},
 '30': {'2544': '4'},
 '31': {'2111': '3'},
 '32': {'3787': '4'},
 '33': {'1247': '3'},
 '34': {'3785': '3'},
 '35': {'1247': '5'},
 '36': {'2985': '4'},
 '37': {'1247': '5'},
 '38': {'1247': '4'},
 '39': {'2396': '5'},
 '40': {'1242': '4'},
 '41': {'2546': '3'},
 '42': {'2986': '4'},
 '43': {'1307': '5'},
 '44': {'2046': '4'},
 '45': {'1247': '3'},
 '46': {'3273': '5'},
 '4

In [ ]:
from math import sqrt
def sim_distance(prefs, p1, p2):
    '''
    Returns a distance-based similarity score for person1 and person2.
    '''

    # Get the list of shared_items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they have no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in prefs[p1] if item in prefs[p2]])
    return 1 / (1 + sqrt(sum_of_squares))

In [52]:
def calculateSimilarItems(prefs, n=10):
    '''
    Create a dictionary of items showing which other items they are
    most similar to.
    '''

    result = {}
    # Invert the preference matrix to be item-centric
    itemPrefs = transformPrefs(prefs)
    c = 0
    for item in itemPrefs:
        # Find the most similar items to this one
        similar_item = topMatches(itemPrefs, item, n=n, similarity=sim_pearson)
        result[item] = similar_item
    return result

In [44]:
def getRecommendations(prefs, person, similarity=sim_pearson):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''

    totals = {}
    simSums = {}
    for other in prefs:
    # Don't compare me to myself
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        # Ignore scores of zero or lower
        if sim <= 0:
            continue
        for item in prefs[other]:
            # Only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                totals.setdefault(item, 0)
                # The final score is calculated by multiplying each item by the
                #   similarity and adding these products together
                totals[item] += prefs[other][item] * sim
                # Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Create the normalized list
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [34]:
def transformPrefs(prefs):
    '''
    Transform the recommendations into a mapping where persons are described
    with interest scores for a given title e.g. {title: person} instead of
    {person: title}.
    '''

    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

In [37]:
class Item:

    def __str__(self):
        if hasattr(self, 'rating'):
            return 'user_id:{0}, movie_id:{1}, rating:{2}, timestamp:{3}'.format(self.user_id, self.movie_id, self.rating, self.timestamp)
        if hasattr(self, 'user_id'):
            return 'user_id:{0}, gender:{1}, age:{2}, occupation:{3}, zip_code:{4}'.format(self.user_id, self.gender, self.age, self.occupation, self.zip_code)
        else:
            return 'movie_id:{0}, title:{1}, genres{2}'.format(self.movie_id, self.title, self.genres)



class Data:

    def __init__(self):
        self.ratings = []
        with open(r'ratings.dat', 'r') as f:
            for l in f:
                item = Item()
                ss = l.split('::')
                ss = [s.strip() for s in ss]
                item.user_id, item.movie_id, item.rating, item.timestamp = ss
                self.ratings.append(item)

        self.users = []
        with open(r'users.dat', 'r') as f:
            for l in f:
                item = Item()
                ss = l.split('::')
                ss = [s.strip() for s in ss]
                item.user_id, item.gender, item.age, item.occupation, item.zip_code = ss
                self.users.append(item)

        self.movies = []
        with open(r'movies.dat', 'r', encoding='ISO-8859-1') as f:
            for l in f:
                item = Item()
                ss = l.split('::')
                ss = [s.strip() for s in ss]
                item.movie_id, item.title, item.genres = ss 
                item.movie_id = item.movie_id.strip()
                self.movies.append(item)

                
data = Data()

d = {}
movie_id_to_movie_title = {}

for movie in data.movies:
    movie_id_to_movie_title[movie.movie_id] = movie.title.encode('cp950', 'ignore').decode('cp950')

for rating in data.ratings:
    if rating.user_id not in d:
        d[rating.user_id] = {}
    d[rating.user_id][movie_id_to_movie_title[rating.movie_id]] = float(rating.rating)





    

In [31]:
def sim_pearson(prefs, p1, p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Sum calculations
    n = len(si)
    # Sums of all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    # Sums of the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])
    # Sum of the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = num / den
    return r

In [54]:
from math import sqrt
sim_pearson(d,"1","2")

0.4166666666666646

In [60]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    '''
    Returns the best matches for person from the prefs dictionary. 
    Number of results and similarity function are optional params.
    '''

    scores = [(similarity(prefs, person, other), other) for other in prefs
              if other != person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [64]:
moviestfp=transformPrefs(d)

In [ ]:
result=calculateSimilarItems(moviestfp,2)

In [ ]:
for i in result.keys():
    print i, result[i]

In [ ]:
def getRecommendedItems_item_based(prefs, similar_item_list, user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}
    # Loop over items rated by this user
    for (item, rating) in userRatings.items():
        # Loop over items similar to this one
        for (similarity, item2) in similar_item_list[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings:
                continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating
            # Sum of all the similarities
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity
    # Divide each total score by total weighting to get an average
    rankings = [(score / totalSim[item], item) for (item, score) in
                scores.items()]
    # Return the rankings from highest to lowest
    rankings.sort()
    rankings.reverse()
    return rankings

In [ ]:
getRecommendedItems_item_based(d,result,"15")